In [36]:
import pandas as pd
import yaml
import requests
import json


In [37]:
secrets_location = "C:\\Users\\lowsona\\Github\\Supporting Files\\secrets.yml"
secrets = open(secrets_location)
secrets = yaml.load(secrets, Loader=yaml.FullLoader)['SharePoint_LGBF']
SP_clientsecret = secrets['client_secret']
SP_tenantid = secrets['tenant_id']
SP_tenant = "stirlingcounciluk"
SP_clientid = secrets['client_id'] + '@' + SP_tenantid


In [38]:
# Establish data to be passed as part of POST request to obtain session token from SharePoint
data = {
    'grant_type': 'client_credentials',
    'resource': "00000003-0000-0ff1-ce00-000000000000/" + SP_tenant + ".sharepoint.com@" + SP_tenantid,
    'client_id': SP_clientid,
    'client_secret': SP_clientsecret,
}

# Establish headers to be passed as part of POST request to obtain session token from SharePoint
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

# Construct url variable for token request
url = "https://accounts.accesscontrol.windows.net/" + \
    SP_tenantid + "/tokens/OAuth/2"

# Send POST request to obtain session token and then load the response into json_data
r = requests.post(url, data=data, headers=headers)
json_data = json.loads(r.text)


In [39]:
# Establish headers to be passed as part of GET request to obtain data from SharePoint list. Using the 'access_token' obtained in the previous step
headers = {
    'Authorization': "Bearer " + json_data['access_token'],
    'Accept': 'application/json;odata=verbose',
    "Accept-Charset": "utf-8",
    'Content-Type': 'application/json;odata=verbose'
}

# Construct url variable for data request
url = "https://" + SP_tenant + \
    ".sharepoint.com/sites/PPA/LGBF/_api/web/lists/GetByTitle('Indicator%20Information')/items"

Items = []

# Send GET request to obtain list data. Extract and normalize this data into a dataframe.


def getItems(url):
    Itemsrequest = requests.get(url, headers=headers)
    Itemsjson = Itemsrequest.json()
    Item = pd.json_normalize(Itemsjson['d']['results'])
    Items.append(Item)
    if '__next' in Itemsjson['d']:
        newurl = Itemsjson['d']['__next']
        getItems(newurl)


getItems(url)

indicatorInformation = pd.DataFrame()
for i in Items:
    indicatorInformation = indicatorInformation.append(i)

indicatorInformation

,FileSystemObjectType,Id,ServerRedirectedEmbedUri,ServerRedirectedEmbedUrl,Title,Code,Code_x0028_SortableA_x002d_Z_x00,ReportingPeriod,MeasureType,NumberFormat,...,GetDlpPolicyTip.__deferred.uri,FieldValuesAsHtml.__deferred.uri,FieldValuesAsText.__deferred.uri,FieldValuesForEdit.__deferred.uri,File.__deferred.uri,Folder.__deferred.uri,LikedByInformation.__deferred.uri,ParentList.__deferred.uri,Properties.__deferred.uri,Versions.__deferred.uri
0,0,18,None,,Net Cost of Waste Collection per Premises,ENV01a,ENV 01a,Annual,Cost,'£ '0,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
1,0,19,None,,Net Cost of Waste Disposal per Premises,ENV02a,ENV 02a,Annual,Cost,'£ '0,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
2,0,20,None,,The % of Total Household Waste Arising that is...,ENV06,ENV 06,Annual,Percentage,0.0%,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
3,0,21,None,,% of Adults Satisfied with Refuse Collection,ENV07a,ENV 07a,3 Year Aggregates,Percentage,0.0%,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
4,0,22,None,,"Net Cost of Street Cleaning per 1,000 Population",ENV03a,ENV 03a,Annual,Cost,"'£ ' #,###",...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0,120,None,,Claimant Count as % of 16-24 Population,ECON12b,ECON 12b,Annual,Percentage,0.0%,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirl

In [40]:
# Drop system columns not needed in dataframe
indicatorInformation = indicatorInformation.drop(columns=['FileSystemObjectType', 'Id', 'ServerRedirectedEmbedUri', 'ServerRedirectedEmbedUrl', 'GetDlpPolicyTip.__deferred.uri', 'FieldValuesAsHtml.__deferred.uri', 'FieldValuesAsText.__deferred.uri', 'FieldValuesForEdit.__deferred.uri', 'File.__deferred.uri', 'Folder.__deferred.uri', 'LikedByInformation.__deferred.uri', 'ParentList.__deferred.uri', 'Properties.__deferred.uri',
                                                 'Versions.__deferred.uri', 'Attachments', 'GUID', '__metadata.id', '__metadata.uri', '__metadata.etag', '__metadata.type', 'FirstUniqueAncestorSecurableObject.__deferred.uri', 'RoleAssignments.__deferred.uri', 'AttachmentFiles.__deferred.uri', 'AttachmentFiles.__deferred.uri', 'ContentType.__deferred.uri', 'ID', 'Modified', 'Created', 'AuthorId', 'EditorId', 'OData__UIVersionString', 'ContentTypeId', 'ComplianceAssetId'])

# Rename columns appropriately
indicatorInformation = indicatorInformation.rename(columns={'Code_x0028_SortableA_x002d_Z_x00': 'Code_Sortable', 'Ranking_x0020_Type': 'Ranking_Type',
                                                   'NumberFormat_x0028_notext_x0029_': "NumberFormat_NoText", "GoldilocksRankingMidpoint_x0028_": "Ranking_GoldilocksMidpoint"})

# Convert column types to ensure that they can be compared to other data later.
indicatorInformation['Code'] = indicatorInformation['Code'].astype(str)
indicatorInformation['Numerator_Match'] = indicatorInformation['Numerator_Match'].astype(
    str)
indicatorInformation['Denominator_Match'] = indicatorInformation['Denominator_Match'].astype(
    str)
indicatorInformation


,Title,Code,Code_Sortable,ReportingPeriod,MeasureType,NumberFormat,YMin,YMax,ISCategory,Committee,...,FormatAxis_Python,AdditionalAxisDenominator_Python,FormatAxis_Plotly_Prefix,FormatAxis_Plotly_Suffix,ImgPxlWidth_Plotly,ImgPxlHeight_Plotly,SubGroup_PythonReport,YMin_Plotly,YMax_Plotly,OData__ColorTag
0,Net Cost of Waste Collection per Premises,ENV01a,ENV 01a,Annual,Cost,'£ '0,40.0,NaN,Environmental Services,Environment and Housing,...,",d",1,£,None,812,305,Waste & Recycling,NaN,NaN,None
1,Net Cost of Waste Disposal per Premises,ENV02a,ENV 02a,Annual,Cost,'£ '0,60.0,NaN,Environmental Services,Environment and Housing,...,",d",1,£,None,812,305,Waste & Recycling,70.00,115.0,None
2,The % of Total Household Waste Arising that is...,ENV06,ENV 06,Annual,Percentage,0.0%,40.0,100.0,Environmental Services,Environment and Housing,...,",.0%",1,None,None,812,305,Waste & Recycling,0.40,0.6,None
3,% of Adults Satisfied with Refuse Collection,ENV07a,ENV 07a,3 Year Aggregates,Percentage,0.0%,50.0,100.0,Environmental Services,Environment and Housing,...,",.0%",1,None,None,812,305,Waste & Recycling,0.60,1.0,None
4,"Net Cost of Street Cleaning per 1,000 Population",ENV03a,ENV 03a,Annual,Cost,"'£ ' #,###",10000.0,NaN,Environmental Services,Environment and Housing,...,s,1000,£,None,800,493,Street Cleaning,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Claimant Count as % of 16-24 Population,ECON12b,ECON 12b,Annual,Percentage,0.0%,NaN,NaN,Economic Development,Finance and Economy,...,",.0%",1,None,None,812,305,Employment,NaN,NaN,None
2,% of Crisis Grant Decisions Within 1 Day,CORP09,CORP 09,Annual,Percentage,0.00%,80.0,100.0,Corporate Services,Community Planning and Regeneration,...,",.0%",1,None,None,931,305,New Indicators,0.85,1.0,None
3,% of CCG Grant Decisions within 15 days,CORP10,CORP 10,Annual,Percentage,0.00%,70.0,100.0,Corporate Services,Community Planning and Regeneration,...,",.0%",1,None,None,931,305,New Indicators,0.75,1.0,None
4,The Proportion of SWF Budget Spent,CORP11,CORP 11,Annual,Percentage,0.00%,70.0,100.0,Corporate Services,Community Planning and Regeneration,...,",.0%",1,None,None,700,493,New Indicators,0.70,1.0,None


In [41]:
indicatorInformation.to_csv(
    "Data Files//Indicator Information.csv", index=False, encoding='utf-8-sig')
